In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import hvplot.pandas
from sklearn.cluster import KMeans

/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Loading data
nhl_df = pd.read_csv('../hockey_starting_data/nhl_analysis_79-11.csv')
nhl_df.head()

,tm_no,year,lgID,tmID,playoff,made_playoff,made_QF,made_CF,made_F,G,...,SHA,PKG,PKC,GF/gm,GA/gm,Goal_spread,PPG/gm,PPG_eff,PKG/gm,PK_eff
0,1,1979,NHL,ATF,PRE,1,0,0,0,80,...,7,52,272,3.52,3.36,0.16,0.64,0.236,0.65,0.191
1,2,1979,NHL,BOS,QF,1,1,0,0,80,...,4,53,312,3.88,2.92,0.96,0.75,0.244,0.66,0.170
2,3,1979,NHL,BUF,SF,1,1,1,0,80,...,4,43,252,3.98,2.51,1.47,0.84,0.244,0.54,0.171
3,4,1979,NHL,CHI,QF,1,1,0,0,80,...,9,56,293,3.01,3.12,-0.11,0.80,0.238,0.70,0.191
4,5,1979,NHL,COR,NaN,0,0,0,0,80,...,3,52,233,2.92,3.85,-0.93,0.66,0.174,0.65,0.223


In [3]:
# Visualize a scatter plot of the data
nhl_df.hvplot.scatter(x="Goal_spread", y="Pts/gm")

:Scatter   [Goal_spread]   (Pts/gm)

In [4]:
selected_columns = ["made_playoff", "Pts/gm", "Goal_spread"]
nhl_playoff_predictor_df = nhl_df[selected_columns]
nhl_playoff_predictor_df.head()

,made_playoff,Pts/gm,Goal_spread
0,1,1.04,0.16
1,1,1.31,0.96
2,1,1.38,1.47
3,1,1.09,-0.11
4,0,0.64,-0.93


In [5]:
# Create a list the number of k-values from 1 to 11
k = list(range(1,11))

# Create an empty list to store the inertia values
inertia = []

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(nhl_playoff_predictor_df)
    inertia.append(k_model.inertia_)

/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samwallach/Documents/BootCamp/Module

In [6]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_original = {"k": k, "inertia": inertia}

# Create a Dataframe with the data to plot the Elbow curve
df_elbow_original = pd.DataFrame(elbow_data_original)

In [7]:
df_elbow_original.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

** Creating a model with 2 clusters

In [8]:
model = KMeans(n_clusters=2, random_state=1)
model

KMeans(n_clusters=2, random_state=1)

In [9]:
model.fit(nhl_playoff_predictor_df)

/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=2, random_state=1)

In [10]:
playoff_prediction = model.predict(nhl_playoff_predictor_df)
playoff_prediction_swapped = [1 if label == 0 else 0 for label in playoff_prediction]

In [11]:
nhl_predictions_df = nhl_playoff_predictor_df.copy()
nhl_predictions_df["playoff_prediction"] = playoff_prediction_swapped
nhl_predictions_df.head()

,made_playoff,Pts/gm,Goal_spread,playoff_prediction
0,1,1.04,0.16,1
1,1,1.31,0.96,1
2,1,1.38,1.47,1
3,1,1.09,-0.11,1
4,0,0.64,-0.93,0


In [12]:
legend_labels = {0: '0 (Missed Playoffs)', 1: '1 (Made Playoffs)'}
nhl_predictions_df["playoff_prediction"] = nhl_predictions_df["playoff_prediction"].replace(legend_labels)

In [13]:
nhl_predictions_df.hvplot.scatter(x='Goal_spread', y="Pts/gm", by="playoff_prediction",
                                  hover_cols=['made_playoff'], legend="bottom_right")

:NdOverlay   [playoff_prediction]
   :Scatter   [Goal_spread]   (Pts/gm,made_playoff)

** Testing the model with 4 clusters

In [14]:
model = KMeans(n_clusters=4, random_state=2)
model

KMeans(n_clusters=4, random_state=2)

In [15]:
model.fit(nhl_playoff_predictor_df)

/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=4, random_state=2)

In [16]:
playoff_prediction = model.predict(nhl_playoff_predictor_df)

In [17]:
nhl_predictions_df = nhl_playoff_predictor_df.copy()
nhl_predictions_df["cluster"] = playoff_prediction
nhl_predictions_df.head()

,made_playoff,Pts/gm,Goal_spread,cluster
0,1,1.04,0.16,2
1,1,1.31,0.96,0
2,1,1.38,1.47,0
3,1,1.09,-0.11,2
4,0,0.64,-0.93,3


In [18]:
nhl_predictions_df.hvplot.scatter(x='Goal_spread', y="Pts/gm", by="cluster",
                                  hover_cols=['made_playoff'], legend="bottom_right")

:NdOverlay   [cluster]
   :Scatter   [Goal_spread]   (Pts/gm,made_playoff)